<a href="https://colab.research.google.com/github/Sanskar6877/ChestXRay_Pneumonia_Detection/blob/main/Model3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# re-size all the images
IMAGE_SIZE = [224, 224]
train_path = '/content/drive/MyDrive/chest_xray/train'
valid_path = '/content/drive/MyDrive/chest_xray/test'

In [4]:
resnet = ResNet50(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

94765736/94765736 [==============================] - 0s 0us/step


In [5]:
for layer in resnet.layers:
    layer.trainable = False


In [6]:
folders = glob('/content/drive/MyDrive/chest_xray/train/*')

In [7]:
x = Flatten()(resnet.output)

In [8]:
prediction = Dense(len(folders), activation='softmax')(x)

# creating a model object
model = Model(inputs=resnet.input, outputs=prediction)

In [9]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                          

In [10]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [11]:


# Use the Image Data Generator to import the images from the dataset
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

# providing the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/chest_xray/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')



test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/chest_xray/test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')



Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [12]:
# fiting the model

r = model.fit(
  training_set,
  validation_data=test_set,
  epochs=5, #10,15,20
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

Epoch 1/5
163/163 [==============================] - 1798s 11s/step - loss: 0.7590 - accuracy: 0.7954 - val_loss: 0.4152 - val_accuracy: 0.8029
Epoch 2/5
163/163 [==============================] - 128s 785ms/step - loss: 0.3518 - accuracy: 0.8547 - val_loss: 0.5394 - val_accuracy: 0.7660
Epoch 3/5
163/163 [==============================] - 128s 787ms/step - loss: 0.3239 - accuracy: 0.8648 - val_loss: 0.3979 - val_accuracy: 0.8221
Epoch 4/5
163/163 [==============================] - 127s 780ms/step - loss: 0.2461 - accuracy: 0.9009 - val_loss: 0.7402 - val_accuracy: 0.7260
Epoch 5/5
163/163 [==============================] - 127s 780ms/step - loss: 0.4967 - accuracy: 0.8551 - val_loss: 0.6950 - val_accuracy: 0.7580
